# 4-2 张量的数学运算
张量的数学运算一般有:
* 标量运算
* 向量运算
* 矩阵运算
* 广播机制

## 一 变量运算
张量的数学运算符可以分为标量运算符、向量运算符、以及矩阵运算符。

加减乘除乘方，以及三角函数，指数，对数等常见函数，逻辑比较运算符等都是标量运算符。

标量运算符的特点是对张量实施逐元素运算。

有些标量运算符对常用的数学运算符进行了重载。并且支持类似numpy的广播特性。

许多标量运算符都在 **tf.math**模块下。

In [1]:
import tensorflow as tf
import numpy as np

In [4]:
a = tf.constant([[1.0, 2], [-3, 4]])

In [5]:
(a >= 2) & (a <= 3)  # 重载了&

<tf.Tensor: shape=(2, 2), dtype=bool, numpy=
array([[False,  True],
       [False, False]])>

In [6]:
(a >= 2) and (a <= 3)  # 不能使用and

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

## 向量运算
向量运算符只在一个特定轴上运算，将一个向量映射到一个标量或者另外一个向量。

许多向量运算符都以**reduce**开头。

In [10]:
a = tf.range(1, 10)
print(tf.reduce_mean(a))
print(tf.reduce_sum(a))
print(tf.reduce_prod(a))  # 各个维度的乘积

tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(45, shape=(), dtype=int32)
tf.Tensor(362880, shape=(), dtype=int32)
tf.Tensor(285, shape=(), dtype=int32)


In [8]:
help(tf.reduce_prod)

Help on function reduce_prod in module tensorflow.python.ops.math_ops:

reduce_prod(input_tensor, axis=None, keepdims=False, name=None)
    Computes the product of elements across dimensions of a tensor.
    
    Reduces `input_tensor` along the dimensions given in `axis`.
    Unless `keepdims` is true, the rank of the tensor is reduced by 1 for each
    entry in `axis`. If `keepdims` is true, the reduced dimensions
    are retained with length 1.
    
    If `axis` is None, all dimensions are reduced, and a
    tensor with a single element is returned.
    
    Args:
      input_tensor: The tensor to reduce. Should have numeric type.
      axis: The dimensions to reduce. If `None` (the default), reduces all
        dimensions. Must be in the range `[-rank(input_tensor),
        rank(input_tensor))`.
      keepdims: If true, retains reduced dimensions with length 1.
      name: A name for the operation (optional).
    
    Returns:
      The reduced tensor.
    
    @compatibility(nump

In [14]:
# bool类型的reduce
p = tf.constant([True, False, False])
q = tf.constant([False, False, True])

print(tf.reduce_all(p))  # 逻辑and
print(tf.reduce_any(q))  # 逻辑or
tf.print(tf.reduce_all(p))  # tf.print输出数字

tf.Tensor(False, shape=(), dtype=bool)
tf.Tensor(True, shape=(), dtype=bool)
0


In [15]:
# tf.foldr函数：This foldr operator repeatedly applies the callable fn to a sequence of elements from last to first, 类似于reduce
s = tf.foldr(lambda a,b:a+b, tf.range(10))  # 使用这个函数实现reduce_sum
print(s)

tf.Tensor(45, shape=(), dtype=int32)


In [17]:
a = tf.range(1, 10)
print(tf.math.cumsum(a))
print(tf.math.cumprod(a))  # 积累乘法

tf.Tensor([ 1  3  6 10 15 21 28 36 45], shape=(9,), dtype=int32)
tf.Tensor([     1      2      6     24    120    720   5040  40320 362880], shape=(9,), dtype=int32)


In [18]:
# arg最大最小值索引
print(tf.argmax(a))
print(tf.argmin(a))

tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)


In [23]:
# tf.math.top_k 进行排序
a = tf.constant([1, 3, 7, 5, 4, 8])
values, indices = tf.math.top_k(a, 3, sorted = True)
print(values)
print(indices)

tf.Tensor([8 7 5], shape=(3,), dtype=int32)
tf.Tensor([5 2 3], shape=(3,), dtype=int32)


## 三 矩阵运算
矩阵必须是二维的。类似tf.constant([1,2,3])这样的不是矩阵。

矩阵运算包括：矩阵乘法，矩阵转置，矩阵逆，矩阵求迹，矩阵范数，矩阵行列式，矩阵求特征值，矩阵分解等运算。

除了一些常用的运算外，大部分和矩阵有关的运算都在**tf.linalg**子包中。

In [24]:
a = tf.constant([[1., 2], [3, 4]])  # 定义矩阵

In [25]:
tf.linalg.inv(a)  # 求逆矩阵要求a必须为浮点数

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-2.0000002 ,  1.0000001 ],
       [ 1.5000001 , -0.50000006]], dtype=float32)>

In [27]:
# 求矩阵的迹
tf.linalg.trace(a)  # 对角线之和

<tf.Tensor: shape=(), dtype=float32, numpy=5.0>

In [28]:
# 求矩阵的范数
tf.linalg.norm(a)

<tf.Tensor: shape=(), dtype=float32, numpy=5.477226>

In [29]:
# 矩阵行列式
tf.linalg.det(a)

<tf.Tensor: shape=(), dtype=float32, numpy=-2.0>

In [30]:
# 求矩阵特征向量
tf.linalg.eigvalsh(a)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-0.8541021,  5.854102 ], dtype=float32)>

### 四，广播机制


TensorFlow的广播规则和numpy是一样的:

* 1、如果张量的维度不同，将维度较小的张量进行扩展，直到两个张量的维度都一样。
* 2、如果两个张量在某个维度上的长度是相同的，或者其中一个张量在该维度上的长度为1，那么我们就说这两个张量在该维度上是相容的。
* 3、如果两个张量在所有维度上都是相容的，它们就能使用广播。
* 4、广播之后，每个维度的长度将取两个张量在该维度长度的较大值。
* 5、在任何一个维度上，如果一个张量的长度为1，另一个张量长度大于1，那么在该维度上，就好像是对第一个张量进行了复制

In [32]:
a = tf.constant([1,2,3])
b = tf.constant([[0,0,0],[1,1,1],[2,2,2]])
print(a.shape)
print(b.shape)
b + a  # 等价于b+tf.broadcast_to(a, b.shape)

(3,)
(3, 3)


<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [2, 3, 4],
       [3, 4, 5]], dtype=int32)>